In [2]:
import dicttoxml

# Example dictionary
data = {
    "name": {
        'surname': 'John',
        'familyname': 'Doe'
    },
    "age": 30,
    "city": "New York"
}

# Convert dictionary to XML
xml_bytes = dicttoxml.dicttoxml(data)
xml_str = xml_bytes.decode()

print(xml_str)


<?xml version="1.0" encoding="UTF-8" ?><root><name type="dict"><surname type="str">John</surname><familyname type="str">Doe</familyname></name><age type="int">30</age><city type="str">New York</city></root>


In [4]:
import xmltodict

# Example dictionary
data = {
    "person": {
        "name": {
        'surname': 'John',
        'familyname': 'Doe'
        },
    "age": 30,
    "city": "New York"
    }
}
    

# Convert dictionary to XML
xml_str = xmltodict.unparse(data, pretty=True)
print(xml_str)


<?xml version="1.0" encoding="utf-8"?>
<person>
	<name>
		<surname>John</surname>
		<familyname>Doe</familyname>
	</name>
	<age>30</age>
	<city>New York</city>
</person>


In [60]:
import os
from bs4 import BeautifulSoup

dir = '/home/alp/Studium/Text Technology/llm-chatbot-python/data/pokewiki_data'

for filename in sorted(os.listdir(dir)):
    filepath = os.path.join(dir, filename)
    with open(filepath, 'r', encoding='utf-8') as f:
        html = f.read()
        break

pokewiki_soup = BeautifulSoup(html, 'html.parser')

In [7]:
dir = '/home/alp/Studium/Text Technology/llm-chatbot-python/data/bisafans_data'

for filename in sorted(os.listdir(dir)):
    filepath = os.path.join(dir, filename)
    with open(filepath, 'r', encoding='utf-8') as f:
        html = f.read()
        break

bisafans_soup = BeautifulSoup(html, 'html.parser')

# Introduction and Biology

In [66]:
import re

def extract_p_text_between_tags(start_tag, end_tag, listlike=False):
    # Ensure both tags are valid BeautifulSoup Tag objects
    if not start_tag or not end_tag:
        return "One or both tags not found."

    # Find all <p> tags between the start_tag and end_tag
    current_tag = start_tag
    p_texts = []
    while current_tag and current_tag != end_tag:
        if current_tag.name == 'p':
            p_texts.append(current_tag.get_text())
        current_tag = current_tag.find_next()

    if listlike:
        clean_list = [re.sub(r'\[\d+\]', '', element.strip().replace('\u2060', ''))
                      for element in p_texts if len(element.strip()) > 0]
        return clean_list


    # Combine all <p> tag texts
    info_text = ' '.join(p_texts)

    # Clean up the resulting text
    info_text = ' '.join(info_text.split()).replace('\u2060', '')
    cleaned_biology = re.sub(r'\[\d+\]', '', info_text)

    return cleaned_biology

intro_first_tag = pokewiki_soup.find('table', {'class':'infobox-pokemon'})
intro_end_tag = pokewiki_soup.find('div', {'id':'toc'})
intro = extract_p_text_between_tags(intro_first_tag, intro_end_tag)

print(intro)


Bisasam ist ein Pokémon der Typen Pflanze und Gift und existiert seit der ersten Spielgeneration. Es ist die erste Entwicklungsstufe von Bisaknosp und Bisaflor. Neben Glumanda und Schiggy ist Bisasam eines der Starter-Pokémon aus Kanto. Als Starter-Pokémon ist es in Pokémon Rot, Blau, Feuerrot und Blattgrün erhältlich und in Pokémon X und Y ist es eines der von Professor Platan erhältlichen Pokémon. Außerdem kann man es in Pokémon Mystery Dungeon: Team Blau und Rot, Erkundungsteam Zeit, Dunkelheit und Himmel sowie in Pokémon Super Mystery Dungeon und Pokémon Mystery Dungeon: Retterteam DX als spielbaren Charakter oder als Partner erhalten. Im Anime und im Manga Pocket Monsters: Dengeki Pikachu nimmt es als Ashs Bisasam eine wichtige Rolle ein. Weitere größere Auftritte besitzt es als Maikes Bisasam im Anime und als Rots Bisasam namens Bisa im „Pocket Monsters SPECIAL“-Manga.


# Anime

In [71]:
biology_first_tag = pokewiki_soup.find('span', {'id':'Spezies'})
if 'id="In_den_Hauptspielen"' in html:
    biology_end_tag = pokewiki_soup.find('span', {'id':'In_den_Hauptspielen'})
else:
    biology_end_tag = pokewiki_soup.find('span', {'id':'In_den_Spielen'})

biologies = extract_p_text_between_tags(biology_first_tag, biology_end_tag, True)


In [72]:
biologies

['Bisasam ist ein kleines, vierbeiniges und reptilienähnliches Pokémon. Es trägt einen hellgrünen Samen auf dem Rücken und sein Körper ist in einem ein klein wenig helleren Grünton gefärbt. Zusätzlich sind auf seiner Haut einzelne, dunkelgrüne Flecken zu erkennen. Das Pflanze-Gift-Pokémon besitzt einen breiten Kopf, der von zwei abgerundeten, dreieckigen Ohren gekrönt wird. Die großen Augen bestehen aus weißer Lederhaut und rot-weißen Pupillen. Die beiden kleinen Nasenlöcher befinden sich direkt über dem sehr breiten Maul. In der schillernden Form erscheint Bisasam komplett in einem leuchtend hellen Grün. Lediglich die Knolle weist ein dunkleres Grün als gewöhnlich auf.',
 'Bisasams Knolle ist der Ausgangspunkt für zahlreiche Fähigkeiten. Sie ermöglicht es dem Pokémon Sonnenlicht zu absorbieren und dieses dort zu speichern. Dadurch kann Bisasam Attacken wie Solarstrahl, Synthese und Wachstum erlernen. Da Bisasam seine Knolle kurzzeitig an der Spitze öffnen kann, kann es die in der Knol

In [57]:
def get_data_from_table(soup, table_id, desired_data):
    table = soup.find_all('dl', {'class':'dl-horizontal'})[table_id]
    for entry in table.find_all('dt'):
        if entry.text != desired_data:
            continue
        
        return entry.find_next().text
    
category = get_data_from_table(bisafans_soup, 0, 'Art')


In [112]:
def get_appearances(soup):
    appearances = {}

    for title in soup.find_all('h4'):
        if title.text != 'Serie':
            continue
        
        appearances[title.text] = []

        for list_element in title.find_next().find_all('li')[1::2]:
            le_text = list_element.text.strip()
            le_text = le_text[le_text.find(' ')+1:]
            if le_text not in appearances[title.text]:
                appearances[title.text].append(le_text)

    for title in soup.find_all('h4'):
        if title.text != 'Filme und Spezialfilme':
            continue
        
        appearances[title.text] = []

        for list_element in title.find_next().find_all('li'):
            le_text = list_element.text.strip().split('\n')[0]
            if le_text not in appearances[title.text]:
                appearances[title.text].append(le_text)

    return appearances

appearances = get_appearances(bisafans_soup)


In [113]:
appearances

{'Serie': ['Episode 1: Pika-Pikachu',
  'Episode 10: Bisasam und das versteckte Idyll',
  'Episode 17: Insel der Giganten',
  'Episode 18: „Schönheit und der Strand!“',
  'Episode 24: Alpollo vs. Kadabra',
  'Episode 26: Im Reich der Düfte',
  'Episode 31: Das Digda-Problem',
  'Episode 32: Die Arena der Ninja',
  'Episode 34: Kangama Kid',
  'Episode 36: Fahrrad-Gang',
  'Episode 41: Duell in Dark City',
  'Episode 43: Angriff der Kokoweis',
  'Episode 50: Kampf um Togepi',
  'Episode 51: Der mysteriöse Garten',
  'Episode 52: Das Prinzessinnenfest',
  'Episode 53: Katzenjammer und Heldengeschichten',
  'Episode 62: Die Piepi Legende',
  'Episode 66: Verirrt im Schneesturm',
  'Episode 70: Terror im Treibhaus',
  'Episode 79: Hopp oder Top – Die Runde 4',
  'Episode 80: Echte Freundschaft!',
  'Episode 83: Panik-Party',
  'Episode 84: Flug des Grauens',
  'Episode 95: Abenteuer auf der Insel Navel',
  'Episode 100: Ferientag mit Hindernissen',
  'Episode 104: Gefahr im Kanal',
  'Epis

# Trivia


In [73]:
def get_trivias(soup):
    for title in soup.find_all('h3'):
        if title.text != 'Trivia':
            continue
        return title.find_next().text.strip().split('\n')
    
trivias = get_trivias(bisafans_soup)

In [74]:
trivias

['In den englischen Versionen von Schwarz und Weiss konnten Bisasam und seine Entwicklungen nicht ohne Spitznamen über die GTS getauscht werden, da ihr englischer Name das deutsche Schimpfwort "Sau" enthält.',
 'In Generation 6 kann man alle Starterpokémon erhalten: In XY die der sechsten und ersten Generation, in ORAS die der zweiten, dritten, vierten und fünften Generation.',
 'In der ersten Folge des Pokémon-Animes stehen auf den Pokébällen der Starter-Pokemon ihre japanischen Namen. Eine Szene mit allen drei Bällen samt Beschriftungen hat es auch in die westliche Fassung geschafft.',
 'Bisasam ist neben Bauz der einzige Starter der Hauptreihe, der von Beginn an einen Dual-Typ ( ) hat.',
 'Alle Entwicklungsstufen von Bisasam kommen in der Super Smash Bros.-Reihe vor.',
 'Es wurde 2001 auf den Niue-Dollar gedruckt.']

# Game

## Game

In [120]:
import xmltodict

# Example dictionary
data = {
    "Pokemon": {
    "Name": 'Bisasam',
    "ID": 1,
    "Types": {
        'EnglishType': {'Type': ['Grass','Poison']},
        'GermanType': {'Type': ['Pflanze','Gift']},
    },
    'Bisafans': {
        "Introduction": intro,
        "Game": {

        },
        "Anime": {
            "Biology": {"P":[biology.replace('\n', ' ') for biology in biologies]},
            "category": category+'-Pokemon',
            "Appearances": {
                "Series": {'Episode': [episode for episode in appearances['Serie']]},
                "Films": {'Movie': [movie for movie in appearances['Filme und Spezialfilme']]}
            },
            "Trivias": {'Trivia': [trivia for trivia in trivias]},
        }
        
    }
    
    }
}
    

# Convert dictionary to XML
xml_str = xmltodict.unparse(data, pretty=True)
print(xml_str)

<?xml version="1.0" encoding="utf-8"?>
<Pokemon>
	<Name>Bisasam</Name>
	<ID>1</ID>
	<Types>
		<EnglishType>
			<Type>Grass</Type>
			<Type>Poison</Type>
		</EnglishType>
		<GermanType>
			<Type>Pflanze</Type>
			<Type>Gift</Type>
		</GermanType>
	</Types>
	<Bisafans>
		<Introduction>Bisasam ist ein Pokémon der Typen Pflanze und Gift und existiert seit der ersten Spielgeneration. Es ist die erste Entwicklungsstufe von Bisaknosp und Bisaflor. Neben Glumanda und Schiggy ist Bisasam eines der Starter-Pokémon aus Kanto. Als Starter-Pokémon ist es in Pokémon Rot, Blau, Feuerrot und Blattgrün erhältlich und in Pokémon X und Y ist es eines der von Professor Platan erhältlichen Pokémon. Außerdem kann man es in Pokémon Mystery Dungeon: Team Blau und Rot, Erkundungsteam Zeit, Dunkelheit und Himmel sowie in Pokémon Super Mystery Dungeon und Pokémon Mystery Dungeon: Retterteam DX als spielbaren Charakter oder als Partner erhalten. Im Anime und im Manga Pocket Monsters: Dengeki Pikachu nimmt es als 